In [17]:
%%writefile cython_fidelity.pyx
import numpy as np
def cython_fidelity(double[:] x, double[:] y):
  cdef int n = x.shape[0]
  cdef int n2 = x.shape[0] // 2
  cdef double res1 = 0
  cdef double res2 = 0
  for i in range(n2):
    res1 += (x[i] * y[i]) + (x[n2 + i] * y[n2 + i])
    res2 += (x[n2 + i] * y[i]) - (x[i] * y[n2 + i])
  return 1 - (res1 * res1) - (res2 * res2)

Overwriting cython_fidelity.pyx


In [18]:
%%writefile setup.py
from setuptools import setup
from Cython.Build import cythonize
import numpy

setup(
    ext_modules = cythonize("cython_fidelity.pyx"),
    include_dirs=[numpy.get_include()]
)

Overwriting setup.py


In [19]:
!python setup.py build_ext --inplace

Compiling cython_fidelity.pyx because it changed.
[1/1] Cythonizing cython_fidelity.pyx
/usr/local/lib/python3.11/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/cython_fidelity.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: cython_fidelity.pyx:8:15: Index should be typed for more efficient access
performance hint: cython_fidelity.pyx:8:22: Index should be typed for more efficient access
performance hint: cython_fidelity.pyx:8:34: Index should be typed for more efficient access
performance hint: cython_fidelity.pyx:8:46: Index should be typed for more efficient access
performance hint: cython_fidelity.pyx:9:18: Index should be typed for more efficient access
performance hint: cython_fidelity.pyx:9:27: Index should be typed for more efficient access
performance hint: cython_fidelity.pyx:9:36: Index should be typed for more eff

In [20]:
import numpy as np
import math
import random
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from cython_fidelity import cython_fidelity

### Functions

In [21]:
#Return a single qubit rotation matrix

# 1. Theta: Angle of rotation

def RotMatCreate(Theta):

  return(np.matrix([[math.cos(Theta), (-1 * math.sin(Theta))], [math.sin(Theta), math.cos(Theta)]]))

In [22]:
#Return matrix that rotates about the z axis in Bloch sphere

# 1. Theta: Angle of rotation

def ZRot(Theta):

  return(np.matrix([[math.cos(Theta/2) - (1j * math.sin(Theta/2)), 0],[0, math.cos(Theta/2) + (1j * math.sin(Theta/2))]]))


In [23]:
# This function generates separable pure states.

# 1. QubitNo: Number of qubits

def GenSepState(QubitNo):

  FinalState = np.matrix([[1]])

  for i in range(QubitNo):
    FinalState = np.kron(FinalState,(2*np.random.random([2,1])-1 + 2j*np.random.random([2,1])-1))

  FinalState = FinalState/np.linalg.norm(FinalState)
  return FinalState

In [24]:
# This function generates entangled pure states.

# 1. QubitNo: Number of qubits.
# 2. IsMaxEnt: Enter 1 if the state should be maximally entangled.

def GenEntState(QubitNo, IsMaxEnt):

  if IsMaxEnt == 1:

    FinalState = np.zeros([2**QubitNo, 1])
    FinalState[0,0] = 1
    FinalState[-1,0] = 1

    RotMat = np.matrix([[1]])
    Thetas = (2*math.pi) * np.random.random(QubitNo)
    Phis = (math.pi) * np.random.random(QubitNo)

    Ry = [RotMatCreate(x/2) for x in Thetas]
    Rz = [ZRot(y) for y in Phis]
    MatY = np.matrix([[1]])
    MatZ = np.matrix([[1]])

    for i in range(QubitNo):
      MatY = np.kron(MatY, Ry[i])
      MatZ = np.kron(MatZ, Rz[i])

    FinalState = np.matmul(MatZ,np.matmul(MatY,FinalState))

  else:

    Re = (2 * np.random.random([2**QubitNo,1])) - 1
    Im = (2 * np.random.random([2**QubitNo,1])) - 1
    FinalState = Re + 1j*Im


  FinalState = FinalState/np.linalg.norm(FinalState)
  return(FinalState)

In [25]:
#Generate NSep separable states

# 1. NSep: Number of separable states to be generated.
# 2. QubitNo: Number of qubits
# 3. Label: The label attached to the states for classification purposes.

def GenSepData(NSep, QubitNo, Label):
  SepData = np.zeros([NSep, 2**QubitNo]) + 1j*np.zeros([NSep, 2**QubitNo])
  for i in range(NSep):
    temp = GenSepState(QubitNo)
    SepData[i,:] = temp.T
  SepData = np.concatenate([SepData, Label*np.ones((NSep,1))], axis=1)
  return SepData

In [26]:
#Generate 2 entangled states among 3 qubits.

# 1. QubitsToEntangle: 12 to entangle first 2 qubits
#                      23 to entangle last 2 qubits
#                      13 to entangle first and last qubits

def GenEntState2(QubitsToEntangle):
  if QubitsToEntangle == 12:
    return np.kron(GenEntState(2, 0), GenSepState(1)).T

  if QubitsToEntangle == 23:
    return np.kron(GenSepState(1), GenEntState(2, 0)).T

  if QubitsToEntangle == 13:
    temp = np.kron(GenSepState(1), GenEntState(2, 0))
    temp = np.reshape(np.array(temp.T), (2,2,2))
    temp = np.ndarray.transpose(temp, [1,0,2])
    temp = np.matrix(np.reshape(temp, (1,8)))
    return temp

In [27]:
# Generate and return NEnt number of entangled states.
# These states would be fully entangled, that is, no bipartition of the qubits would result in two separable states.

# 1. NEnt: Number of states to be generated.
# 2. QubitNo: Number of qubits.
# 3. IsMaxEnt: Enter 1 if the states should be maximally entangled.
# 4. Label: The label attached to the states for classification purposes.

def GenEntData(NEnt, QubitNo, IsMaxEnt, Label):
  EntData = np.zeros([NEnt, 2**QubitNo]) + 1j*np.zeros([NEnt, 2**QubitNo])

  for i in range(NEnt):
    temp = GenEntState(QubitNo, IsMaxEnt)
    EntData[i,:] = temp.T
  EntData = np.concatenate([EntData, Label*np.ones((NEnt,1))], axis=1)
  return EntData

In [28]:
# This function can generate a dataset of entangled states.

# 1. N: Number of states to be generated.
# 2. QubitsToEntangle: 12 to entangle first 2 qubits
#                      23 to entangle last 2 qubits
#                      13 to entangle first and last qubits
# 3. Label: A label for the entangled states. This label helps while providing the data to the KNN model. It can be any integer.

def GenMultiEntData(N, QubitsToEntangle, Label):
  Data = np.zeros([N, 8]) + 1j*np.zeros([N, 8])
  for i in range(N):
    Data[i,:] = GenEntState2(QubitsToEntangle)
  Data = np.concatenate([Data, Label*np.ones((N,1))], axis=1)
  return Data

### Classical KNN



In [29]:
NSep = 100000
NEnt = 100000
n = 2

DataSep = GenSepData(NSep, n, 0)
Data12 = GenEntData(NEnt, n, 0, 1)
Data = np.concatenate([DataSep, Data12], axis = 0)

np.random.shuffle(Data)

DataX = Data[:,:-1]
DataY = Data[:,-1]
proportion = 400
DataSplitX = DataX
DataSplitY = np.real(DataY)
DataSplitX = np.concatenate([np.real(DataSplitX), np.imag(DataSplitX)], axis = 1)
XTrain, YTrain, XTest, YTest = DataSplitX[:-int((NSep+NEnt)/proportion)], DataSplitY[:-int((NSep+NEnt)/proportion)], DataSplitX[-int((NSep+NEnt)/proportion):], DataSplitY[-int((NSep+NEnt)/proportion):]
NeighComp = KNeighborsClassifier(n_neighbors=3, metric = cython_fidelity)
NeighComp.fit(XTrain, YTrain)
Ans = NeighComp.predict(XTest)
print("Accuracy = ",accuracy_score(YTest, Ans))
confusion_matrix(YTest, Ans)

Accuracy =  0.982


array([[246,   0],
       [  9, 245]])

#### 2 qubit with maximally entangled states

In [30]:
NSep = 100000
NEnt = 100000
n = 2

DataSep = GenSepData(NSep, n, 0)
Data12 = GenEntData(NEnt, n, 1, 1)
Data = np.concatenate([DataSep, Data12], axis = 0)

np.random.shuffle(Data)

DataX = Data[:,:-1]
DataY = Data[:,-1]
proportion = 400
DataSplitX = DataX
DataSplitY = np.real(DataY)
DataSplitX = np.concatenate([np.real(DataSplitX), np.imag(DataSplitX)], axis = 1)
XTrain, YTrain, XTest, YTest = DataSplitX[:-int((NSep+NEnt)/proportion)], DataSplitY[:-int((NSep+NEnt)/proportion)], DataSplitX[-int((NSep+NEnt)/proportion):], DataSplitY[-int((NSep+NEnt)/proportion):]
NeighComp = KNeighborsClassifier(n_neighbors=3, metric = cython_fidelity)
NeighComp.fit(XTrain, YTrain)
Ans = NeighComp.predict(XTest)
print("Accuracy = ",accuracy_score(YTest, Ans))
confusion_matrix(YTest, Ans)

Accuracy =  1.0


array([[241,   0],
       [  0, 259]])

#### 3 qubit

In [31]:
N = 100000
DataSep = GenSepData(N, 3, 0)
Data12 = GenMultiEntData(N, 12, 12)
Data23 = GenMultiEntData(N, 23, 23)
Data13 = GenMultiEntData(N, 13, 13)
Data123 = GenEntData(N, 3, 0, 123)

Data = np.concatenate([DataSep, Data12, Data23, Data13, Data123], axis = 0)
np.random.shuffle(Data)

DataX = Data[:,:-1]
DataY = Data[:,-1]
proportion = 1000
DataSplitX = DataX
DataSplitY = np.real(DataY)
DataSplitX = np.concatenate([np.real(DataSplitX), np.imag(DataSplitX)], axis = 1)
XTrain, YTrain, XTest, YTest = DataSplitX[:-int((N * 5)/proportion)], DataSplitY[:-int((N * 5)/proportion)], DataSplitX[-int((N * 5)/proportion):], DataSplitY[-int((N * 5)/proportion):]
NeighComp = KNeighborsClassifier(n_neighbors=10, metric = cython_fidelity)
NeighComp.fit(XTrain, YTrain)
Ans = NeighComp.predict(XTest)
print("Accuracy = ",accuracy_score(YTest, Ans))
confusion_matrix(YTest, Ans)

Accuracy =  0.902


array([[102,   1,   0,   0,   0],
       [  7, 104,   1,   1,   0],
       [  7,   0,  86,   1,   0],
       [  5,   2,   1,  93,   0],
       [  0,   8,  10,   5,  66]])